In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv("featuresets/1/train_data_cleaned.csv",index_col=0)
test = pd.read_csv("featuresets/1/test_data_cleaned.csv",index_col=0)

In [3]:
shift = 200
x_train = train.drop("loss",axis=1)
y = np.log(train['loss']+shift)
x_test = test

## XGboost

In [4]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

/home/achal/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
RANDOM_STATE = 20
xgb_params = {
    'seed': 0,
    'colsample_bytree': 0.7,
    'silent': 1,
    'subsample': 0.7,
    'learning_rate': 0.070,
    'objective': 'reg:linear',
    'max_depth': 6,
    'num_parallel_tree': 1,
    'min_child_weight': 1,
    'eval_metric': 'mae',
    'latent' : RANDOM_STATE
}

xgtrain = xgb.DMatrix(x_train, label=y)
xgtest = xgb.DMatrix(x_test)

In [11]:
def xg_eval_mse(yhat, dtrain):
    y = dtrain.get_label()
    return('mae', mean_absolute_error(np.exp(y), np.exp(yhat)-shift))

In [12]:
res = xgb.cv(xgb_params, xgtrain, num_boost_round=1000, nfold=5, seed=RANDOM_STATE, stratified=False,early_stopping_rounds=50, verbose_eval=10, show_stdv=True, feval=xg_eval_mse, maximize=False)

[0]	train-mae:3434.62+4.1038	test-mae:3434.62+16.4156
[10]	train-mae:3343.3+4.07081	test-mae:3343.3+16.3218
[20]	train-mae:2905.65+3.66776	test-mae:2905.73+15.7789
[30]	train-mae:2209.38+2.91214	test-mae:2210.68+14.5543
[40]	train-mae:1658.38+1.96	test-mae:1663.44+10.7024
[50]	train-mae:1383.73+1.90538	test-mae:1393.56+8.23169
[60]	train-mae:1266.81+1.46301	test-mae:1280.68+7.36851
[70]	train-mae:1214.65+1.60021	test-mae:1231.54+6.92545
[80]	train-mae:1188.32+1.47361	test-mae:1208.11+6.94251
[90]	train-mae:1173.41+1.57436	test-mae:1195.66+6.73493
[100]	train-mae:1163.63+1.69404	test-mae:1187.89+6.66357
[110]	train-mae:1156.78+1.68011	test-mae:1183.25+6.72226
[120]	train-mae:1151.18+1.65166	test-mae:1179.46+6.79711
[130]	train-mae:1146.85+1.59042	test-mae:1177.03+6.75342
[140]	train-mae:1142.7+1.43721	test-mae:1174.95+6.73184
[150]	train-mae:1139.21+1.28018	test-mae:1173.31+6.90938
[160]	train-mae:1135.92+1.3926	test-mae:1171.94+6.79353
[170]	train-mae:1132.79+1.57539	test-mae:1170.61+6

In [13]:
best_nrounds = res.shape[0] - 1
cv_mean = res.iloc[-1, 0]
cv_std = res.iloc[-1, 1]
print('CV-Mean: {0}+{1}'.format(cv_mean, cv_std))

CV-Mean: 1160.3170164+6.918289952706379


In [15]:
gbdt = xgb.train(xgb_params, xgtrain, best_nrounds)

KeyboardInterrupt: 

In [18]:
xgb.plot_importance()

<function xgboost.plotting.plot_importance>